In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import tensorflow as tf

In [23]:
WINDOW_SIZE = 10
WINDOW_STEP_SIZE = 1

In [24]:
ROOTDIR_DATASET = './KukaVelocityDataset'
kuka_column_names_path = os.path.join(ROOTDIR_DATASET, 'KukaColumnNames.npy')
kuka_column_names = np.load(kuka_column_names_path)
kuka_column_names.reshape((1, -1))

array([['action', 'machine_nameKuka Robot_apparent_power',
        'machine_nameKuka Robot_current',
        'machine_nameKuka Robot_frequency',
        'machine_nameKuka Robot_phase_angle',
        'machine_nameKuka Robot_power',
        'machine_nameKuka Robot_power_factor',
        'machine_nameKuka Robot_reactive_power',
        'machine_nameKuka Robot_voltage', 'sensor_id1_AccX',
        'sensor_id1_AccY', 'sensor_id1_AccZ', 'sensor_id1_GyroX',
        'sensor_id1_GyroY', 'sensor_id1_GyroZ', 'sensor_id1_q1',
        'sensor_id1_q2', 'sensor_id1_q3', 'sensor_id1_q4',
        'sensor_id1_temp', 'sensor_id2_AccX', 'sensor_id2_AccY',
        'sensor_id2_AccZ', 'sensor_id2_GyroX', 'sensor_id2_GyroY',
        'sensor_id2_GyroZ', 'sensor_id2_q1', 'sensor_id2_q2',
        'sensor_id2_q3', 'sensor_id2_q4', 'sensor_id2_temp',
        'sensor_id3_AccX', 'sensor_id3_AccY', 'sensor_id3_AccZ',
        'sensor_id3_GyroX', 'sensor_id3_GyroY', 'sensor_id3_GyroZ',
        'sensor_id3_q1', 'sensor_i

In [25]:
kuka_normal_path = os.path.join(ROOTDIR_DATASET, 'KukaNormal.npy')
kuka_normal = np.load(kuka_normal_path)
kuka_normal = np.hstack((kuka_normal, np.zeros(kuka_normal.shape[0]).reshape(-1, 1) ))

kuka_normal = np.vstack((kuka_column_names, kuka_normal))
df_kuka_normal = pd.DataFrame(kuka_normal)
df_kuka_normal.columns = df_kuka_normal.iloc[0]
df_kuka_normal = df_kuka_normal.iloc[1:]

df_kuka_normal.drop("anomaly", axis=1, inplace=True)
df_kuka_normal = df_kuka_normal.reset_index(drop=True)
df_kuka_normal['machine_nameKuka Robot_time'] = df_kuka_normal.index.astype(float) /  df_kuka_normal['machine_nameKuka Robot_frequency'].astype(float)

df_kuka_normal

,action,machine_nameKuka Robot_apparent_power,machine_nameKuka Robot_current,machine_nameKuka Robot_frequency,machine_nameKuka Robot_phase_angle,machine_nameKuka Robot_power,machine_nameKuka Robot_power_factor,machine_nameKuka Robot_reactive_power,machine_nameKuka Robot_voltage,sensor_id1_AccX,...,sensor_id7_AccZ,sensor_id7_GyroX,sensor_id7_GyroY,sensor_id7_GyroZ,sensor_id7_q1,sensor_id7_q2,sensor_id7_q3,sensor_id7_q4,sensor_id7_temp,machine_nameKuka Robot_time
0,0.0,163.162201,0.976771,50.0,320.70929,126.283424,0.773975,-103.307549,228.322586,-0.68994140625,...,1.0009765625,0.0,0.0,0.0,-0.08373313953176298,0.15915735966550562,-0.8742612934258501,0.4509156096497644,180.24,0.00
1,0.0,163.162201,0.976771,50.0,320.70929,126.283424,0.773975,-103.307549,228.322586,-0.689453125,...,1.00146484375,0.0,0.0,0.0,-0.08373313953176298,0.15915735966550562,-0.8742612934258501,0.4509156096497644,180.24,0.02
2,0.0,163.162201,0.976771,50.0,320.70929,126.283424,0.773975,-103.307549,228.322586,-0.68896484375,...,1.00048828125,0.0,0.0,0.0,-0.08373313953176298,0.15915735966550562,-0.8742612934258501,0.4509156096497644,180.24,0.04
3,0.0,163.162201,0.976771,50.0,320.70929,126.283424,0.773975,-103.307549,228.322586,-0.68994140625,...,1.00146484375,0.0,0.0,0.0,-0.08373313953176298,0.15915735966550562,-0.8742612934258501,0.4509156096497644,180.24,0.06
4,0.0,163.162201,0.976771,50.0,320.70929,126.283424,0.773975,-103.307549,228.322586,-0.6875,...,0.9990234375,0.0,0.0,0.0,-0.08373313953176298,0.15915735966550562,-0.8742612934258501,0.4509156096497644,180.24,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233787,0.0,160.462463,0.993556,50.0,321.580872,125.725021,0.783517,-99.697563,231.03653,-0.75439453125,...,1.00048828125,0.0,0.0,0.0,-0.17272378949843972,-0.048043706319791,0.2517675893374837,0.9510370002179456,180.24,4675.74
233788,0.0,160.462463,0.993556,50.0,321.580872,125.725021,0.783517,-99.697563,231.03653,-0.7509765625,...,1.0009765625,0.0,0.0,0.0,-0.17272378949843972,-0.048043706319791,0.2517675893374837,0.9510370002179456,180.24,4675.76
233789,0.0,160.462463,0.993556,50.0,321.580872,125.725021,0.783517,-99.697563,231.03653,-0.75439453125,...,0.99951171875,0.0,0.0,0.0,-0.17272378949843972,-0.048043706319791,0.2517675893374837,0.9510370002179456,180.24,4675.78
233790,0.0,160.462463,0.993556,50.0,321.580872,125.725021,0.783517,-99.697563,231.03653,-0.7626953125,...,1.0,0.0,0.06103515625,0.0,-0.1729321422552059,-0.0481786261720243,0.2516482003486041,0.9510239100130503,180.24,4675.80


In [26]:
df_kuka_normal['action'].astype(float).unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31.])

In [27]:
kuka_slow_path = os.path.join(ROOTDIR_DATASET, 'KukaSlow.npy')
kuka_slow = np.load(kuka_slow_path)

kuka_slow = np.vstack((kuka_column_names, kuka_slow))

df_kuka_slow = pd.DataFrame(kuka_slow)
df_kuka_slow.columns = df_kuka_slow.iloc[0]
df_kuka_slow = df_kuka_slow.iloc[1:]
df_kuka_slow = df_kuka_slow.reset_index(drop=True)
df_kuka_slow.drop("anomaly", axis=1, inplace=True)
df_kuka_slow['machine_nameKuka Robot_time'] = df_kuka_slow.index.astype(float) /  df_kuka_slow['machine_nameKuka Robot_frequency'].astype(float)
df_kuka_slow

,action,machine_nameKuka Robot_apparent_power,machine_nameKuka Robot_current,machine_nameKuka Robot_frequency,machine_nameKuka Robot_phase_angle,machine_nameKuka Robot_power,machine_nameKuka Robot_power_factor,machine_nameKuka Robot_reactive_power,machine_nameKuka Robot_voltage,sensor_id1_AccX,...,sensor_id7_AccZ,sensor_id7_GyroX,sensor_id7_GyroY,sensor_id7_GyroZ,sensor_id7_q1,sensor_id7_q2,sensor_id7_q3,sensor_id7_q4,sensor_id7_temp,machine_nameKuka Robot_time
0,10.0,216.610596,1.201105,50.0,330.759125,189.01265,0.872592,-105.793724,228.445389,-0.6298828125,...,1.0224609375,-0.9765625,-9.033203125,40.22216796875,-0.21068182200865018,0.013827193347123616,-0.336949831014503,0.9175438899467179,180.24,0.000000
1,10.0,216.610596,1.201105,50.0,330.759125,189.01265,0.872592,-105.793724,228.445389,-0.6298828125,...,1.0126953125,-0.18310546875,-2.0751953125,6.7138671875,-0.21132194557528416,0.010657524165009119,-0.31755194499842027,0.924332307519687,180.24,0.020000
2,10.0,216.610596,1.201105,49.951218,330.759125,189.01265,0.872592,-105.793724,228.445389,-0.490234375,...,1.01953125,0.0,-0.18310546875,-0.1220703125,-0.20851756419089468,0.014488422012170703,-0.3164776458502097,0.9252850429616825,180.24,0.040039
3,10.0,216.610596,1.201105,49.951218,330.759125,189.01265,0.872592,-105.793724,228.445389,-0.62353515625,...,0.9892578125,0.06103515625,0.18310546875,-0.30517578125,-0.20604036625573777,0.01736908852966379,-0.31614382591187384,0.9259042950406121,180.24,0.060059
4,10.0,216.610596,1.201105,49.951218,330.759125,189.01265,0.872592,-105.793724,228.445389,-0.591796875,...,0.98779296875,-0.1220703125,0.1220703125,0.67138671875,-0.20396720033750398,0.01972520503749875,-0.3158315901978187,0.9264225300076775,180.24,0.080078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41533,0.0,216.71846,1.265179,49.951218,329.322937,186.394455,0.860077,-110.552055,228.759598,-0.5986328125,...,1.01171875,0.0,-9.82666015625,42.17529296875,-0.15644585522798213,-0.1085471359527849,0.3916207025737444,0.9002085530442819,180.24,831.471217
41534,0.0,216.71846,1.265179,49.951218,329.322937,186.394455,0.860077,-110.552055,228.759598,-0.5986328125,...,0.9912109375,-0.1220703125,-9.70458984375,41.1376953125,-0.15205419305733545,-0.12002028892521356,0.4228920602115566,0.885232714053212,180.24,831.491236
41535,0.0,216.71846,1.265179,49.951218,329.322937,186.394455,0.860077,-110.552055,228.759598,-0.5986328125,...,1.013671875,0.0,-9.6435546875,41.9921875,-0.14758016918857209,-0.1305584989451105,0.45207474710525886,0.8699442482396146,180.24,831.511256
41536,0.0,216.71846,1.265179,49.951218,329.322937,186.394455,0.860077,-110.552055,228.759598,-0.5986328125,...,0.99755859375,-0.48828125,-9.94873046875,40.52734375,-0.1426466455260191,-0.1413067618311141,0.48235496015583823,0.8526535204841641,180.24,831.531275


In [28]:
a: np.ndarray = df_kuka_slow['action'].astype(float).unique()
a.sort()
a

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30., 31.])

In [29]:
class Encoder(tf.keras.Model):
    def __init__(self, window_size, latent_space):
        super(Encoder, self).__init__()
        # TODO: set window size for each layer -> in and out of conv
        self.conv1d_1 = tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='causal')
        self.conv1d_2 = tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='causal')
        self.conv1d_3 = tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='causal')
        self.conv1d_4 = tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='causal')
        self.conv1d_5 = tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='causal')
        self.batch_norm_1 = tf.keras.layers.BatchNormalization()
        self.batch_norm_2 = tf.keras.layers.BatchNormalization()
        self.batch_norm_3 = tf.keras.layers.BatchNormalization()
        self.batch_norm_4 = tf.keras.layers.BatchNormalization()

    def build(self, input_shape):
        super(Encoder, self).build(input_shape)

    def call(self, X):
        X = self.conv1d_1(X)
        X = self.batch_norm_1(X)
        X = tf.nn.leaky_relu(X)

        X = self.conv1d_2(X)
        X = self.batch_norm_2(X)
        X = tf.nn.leaky_relu(X)

        X = self.conv1d_3(X)
        X = self.batch_norm_3(X)
        X = tf.nn.leaky_relu(X)

        X = self.conv1d_4(X)
        X = self.batch_norm_4(X)
        X = tf.nn.leaky_relu(X)

        X = self.conv1d_5(X)

        return X


class Decoder(tf.keras.Model):
    def __init__(self, window_size, latent_space):
        super(Decoder, self).__init__()
        # TODO: set window size for each layer -> in and out of conv
        self.conv_transpose_1d_1 = tf.keras.layers.ConvTranspose1d(filters=32, kernel_size=3, padding='causal')
        self.conv_transpose_1d_2 = tf.keras.layers.ConvTranspose1d(filters=32, kernel_size=3, padding='causal')
        self.conv_transpose_1d_3 = tf.keras.layers.ConvTranspose1d(filters=32, kernel_size=3, padding='causal')
        self.conv_transpose_1d_4 = tf.keras.layers.ConvTranspose1d(filters=32, kernel_size=3, padding='causal')
        self.conv_transpose_1d_5 = tf.keras.layers.ConvTranspose1d(filters=32, kernel_size=3, padding='causal')
        self.batch_norm_1 = tf.keras.layers.BatchNormalization()
        self.batch_norm_2 = tf.keras.layers.BatchNormalization()
        self.batch_norm_3 = tf.keras.layers.BatchNormalization()
        self.batch_norm_4 = tf.keras.layers.BatchNormalization()

    def build(self, input_shape):
        super(Decoder, self).build(input_shape)

    def call(self, X):
        X = self.conv1d_1(X)
        X = self.batch_norm_1(X)
        X = tf.nn.relu(X)

        X = self.conv1d_2(X)
        X = self.batch_norm_2(X)
        X = tf.nn.relu(X)

        X = self.conv1d_3(X)
        X = self.batch_norm_3(X)
        X = tf.nn.relu(X)

        X = self.conv1d_4(X)
        X = self.batch_norm_4(X)
        X = tf.nn.relu(X)

        X = self.conv1d_5(X)

        return X
    

class AutoEncoder(tf.keras.Model):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = Encoder(WINDOW_SIZE, WINDOW_STEP_SIZE)
        self.decoder = Decoder(WINDOW_SIZE, WINDOW_STEP_SIZE)

    def build(self, input_shape):
        super(AutoEncoder, self).build(input_shape)

    def call(self, X):
        latent = self.encoder(X)
        X = self.decoder(latent)

        return X

In [30]:
df_kuka_normal.iloc[:1000]

,action,machine_nameKuka Robot_apparent_power,machine_nameKuka Robot_current,machine_nameKuka Robot_frequency,machine_nameKuka Robot_phase_angle,machine_nameKuka Robot_power,machine_nameKuka Robot_power_factor,machine_nameKuka Robot_reactive_power,machine_nameKuka Robot_voltage,sensor_id1_AccX,...,sensor_id7_AccZ,sensor_id7_GyroX,sensor_id7_GyroY,sensor_id7_GyroZ,sensor_id7_q1,sensor_id7_q2,sensor_id7_q3,sensor_id7_q4,sensor_id7_temp,machine_nameKuka Robot_time
0,0.0,163.162201,0.976771,50.0,320.70929,126.283424,0.773975,-103.307549,228.322586,-0.68994140625,...,1.0009765625,0.0,0.0,0.0,-0.08373313953176298,0.15915735966550562,-0.8742612934258501,0.4509156096497644,180.24,0.000000
1,0.0,163.162201,0.976771,50.0,320.70929,126.283424,0.773975,-103.307549,228.322586,-0.689453125,...,1.00146484375,0.0,0.0,0.0,-0.08373313953176298,0.15915735966550562,-0.8742612934258501,0.4509156096497644,180.24,0.020000
2,0.0,163.162201,0.976771,50.0,320.70929,126.283424,0.773975,-103.307549,228.322586,-0.68896484375,...,1.00048828125,0.0,0.0,0.0,-0.08373313953176298,0.15915735966550562,-0.8742612934258501,0.4509156096497644,180.24,0.040000
3,0.0,163.162201,0.976771,50.0,320.70929,126.283424,0.773975,-103.307549,228.322586,-0.68994140625,...,1.00146484375,0.0,0.0,0.0,-0.08373313953176298,0.15915735966550562,-0.8742612934258501,0.4509156096497644,180.24,0.060000
4,0.0,163.162201,0.976771,50.0,320.70929,126.283424,0.773975,-103.307549,228.322586,-0.6875,...,0.9990234375,0.0,0.0,0.0,-0.08373313953176298,0.15915735966550562,-0.8742612934258501,0.4509156096497644,180.24,0.080000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3.0,182.85611,0.888157,49.951218,331.718079,161.031158,0.880644,-86.625328,228.640778,-1.15478515625,...,0.96630859375,0.18310546875,15.5029296875,-68.42041015625,-0.012764360657293042,0.19107242996968782,-0.9432588500252186,0.2712768686261171,180.24,19.919434
996,3.0,182.85611,0.888157,49.951218,331.718079,161.031158,0.880644,-86.625328,228.640778,-1.0009765625,...,1.05126953125,0.79345703125,16.2353515625,-66.0400390625,0.006952523040431514,0.19711854473146037,-0.9561288207826855,0.21659552122570527,180.24,19.939454
997,3.0,182.85611,0.888157,49.951218,331.718079,161.031158,0.880644,-86.625328,228.640778,-0.7607421875,...,1.078125,0.06103515625,16.29638671875,-67.5048828125,0.02633044568145951,0.20216513482427184,-0.9656332046717677,0.16120942877831815,180.24,19.959473
998,3.0,182.85611,0.888157,49.951218,331.718079,161.031158,0.880644,-86.625328,228.640778,-1.24462890625,...,0.9794921875,0.0,10.8642578125,-43.212890625,0.04216351907667466,0.20523992483268538,-0.9713555989782695,0.1121031277320654,180.24,19.979493


In [31]:
def sliding_window(data, window_size, step_size):
    windows = []
    for i in range(0, data.shape[0] - window_size + 1, step_size):
        windows.append(data.iloc[i:i + window_size].to_numpy())
    
    return np.array(windows)

sliding_window(df_kuka_normal, WINDOW_SIZE, WINDOW_STEP_SIZE).shape

(233783, 10, 87)

In [32]:
"""
autoencoder: AutoEncoder = AutoEncoder()
autoencoder.build(train_data.shape)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

autoencoder.compile(optimizer=optimizer, loss='mse')
autoencoder.fit(train_data, train_data, batch_size=1024, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS, callbacks=[callback])
autoencoder.summary()
"""

"\nautoencoder: AutoEncoder = AutoEncoder()\nautoencoder.build(train_data.shape)\n\noptimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)\ncallback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)\n\nautoencoder.compile(optimizer=optimizer, loss='mse')\nautoencoder.fit(train_data, train_data, batch_size=1024, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS, callbacks=[callback])\nautoencoder.summary()\n"